In [1]:
import tweets_processor
import importlib
import vocabulary
import utils
import models
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import patched_numpy_io
from sklearn.preprocessing import LabelEncoder

/Users/divyagorantla/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/divyagorantla/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
importlib.reload(tweets_processor)
#tweets_text, tweets_place = tweets_processor.get_tweets_with_location()
tweets_text, tweets_place = tweets_processor.get_tweets_from_csv()
#print(tweets_text[-1],tweets_place[-1])

In [3]:
# split the data
######TODO: Split into train, dev, test -use kfold cross validation #########
train_data, test_data, train_labels, test_labels = train_test_split(tweets_text, tweets_place, test_size=0.33, random_state=0)

In [4]:
# processing the tweets
def get_tokenized_tweets(tweets):
    processed_tweets = []
    tokenized_tweets = []
    tokens = []
    for tweet in tweets:
        processed_tweet = tweets_processor.preprocessor(tweet)
        processed_tweets.append(processed_tweet)
        # pass through tokenizer
        tokenized_tweet = tweets_processor.tokenizer(processed_tweet)
        # for each tweet get the tokens
        tokenized_tweets.append(tokenized_tweet)
        # create list of all tokens
        tokens.extend(tokenized_tweet)
    return tokenized_tweets, tokens
#print(tokenized_tweets)

In [5]:
# processing tokens to replace anything not in vocab with <unk>
def process_tokens_unk(tokens):
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in tokens], dtype=object)

In [6]:
# get train tokens and vocabulary tokens from trian data set
train_tokenized_tweets, train_tokens = get_tokenized_tweets(train_data)    

In [7]:
# create vocabulary
vocab = vocabulary.Vocabulary(utils.canonicalize_word(w,digits=False) for w in train_tokens)
print(vocab.size)

13711


In [8]:
# process the train_tokenized_tweets
processed_train_tokenized_tweets = []
for tokenized_tweet in train_tokenized_tweets:
    processed_unk_tweet = process_tokens_unk(tokenized_tweet)
    processed_train_tokenized_tweets.append(processed_unk_tweet)
    
train_tokenized_tweets_ids = []
for tokenized_tweet in processed_train_tokenized_tweets:
    tokenized_tweet_id = vocab.words_to_ids(tokenized_tweet)
    train_tokenized_tweets_ids.append(tokenized_tweet_id)
    #print(tokenized_tweet, tokenized_tweet_id)
    

In [9]:
# get test tokens
test_tokenized_tweets, test_tokens = get_tokenized_tweets(test_data)
# process the test_tokenized_tweets
processed_test_tokenized_tweets = []
for tokenized_tweet in test_tokenized_tweets:
    processed_unk_tweet = process_tokens_unk(tokenized_tweet)
    processed_test_tokenized_tweets.append(processed_unk_tweet)
    
test_tokenized_tweets_ids = []
for tokenized_tweet in processed_test_tokenized_tweets:
    tokenized_tweet_id = vocab.words_to_ids(tokenized_tweet)
    test_tokenized_tweets_ids.append(tokenized_tweet_id)

In [10]:
# encode train labels
#Converting string to integers

le = LabelEncoder()
train_encoded_labels = le.fit_transform(train_labels)
print(train_encoded_labels)

print(le.inverse_transform(train_encoded_labels))

[ 4  4 13 ...  6 18 13]
['cincinnati' 'cincinnati' 'new york' ... 'houston' 'san francisco'
 'new york']


/Users/divyagorantla/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [12]:
# padding sequences of smaller
train_x, train_ns = utils.pad_np_array(train_tokenized_tweets_ids, max_len=50, pad_id=0)
#print(train_x)
#print(train_tokenized_tweets_ids)


In [13]:
# passing through NBOW model
# training
################### TODO: graph using tensorborad ################
train_labels_array = np.array(train_encoded_labels, dtype=np.int32)
model_params = dict(V=vocab.size, embed_dim=50, hidden_dims=[25], num_classes=len(train_labels_array),
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

model = tf.estimator.Estimator(model_fn=models.classifier_model_fn, 
                               params=model_params)

train_params = dict(batch_size=32, total_epochs=20, eval_every=2)

train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_ns}, y=train_labels_array,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):
    # Train for a few epochs, then evaluate on dev
    model.train(input_fn=train_input_fn)
    #### TODO: do the evaluation on dev ###


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/68/1f0ftm9d7_b6h52231rtg7mh0000gn/T/tmpy16h5yus', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10f3443c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 i

INFO:tensorflow:Loss for final step: 2.3906102.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/68/1f0ftm9d7_b6h52231rtg7mh0000gn/T/tmpy16h5yus/model.ckpt-2933
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2933 into /var/folders/68/1f0ftm9d7_b6h52231rtg7mh0000gn/T/tmpy16h5yus/model.ckpt.
INFO:tensorflow:loss = 2.4190812, step = 2934
INFO:tensorflow:global_step/sec: 148.579
INFO:tensorflow:loss = 2.858951, step = 3034 (0.675 sec)
INFO:tensorflow:global_step/sec: 154.536
INFO:tensorflow:loss = 2.5664449, step = 3134 (0.647 sec)
INFO:tensorflow:global_step/sec: 164.22
INFO:tensorflow:loss = 2.4705966, step = 3234 (0.610 sec)
INFO:tensorflow:global_step/sec: 143.558
INFO:tensorflow:loss = 2.3927746, step = 3334 (0.695 sec)
INFO:tensorflow:Saving checkpoints

In [14]:
# get test inputs
test_x, test_ns = utils.pad_np_array(test_tokenized_tweets_ids, max_len=140, pad_id=0)

# encode test labels
le = LabelEncoder()
test_encoded_labels = le.fit_transform(test_labels)


In [15]:
# evaluate model
test_labels_array = np.array(test_encoded_labels, dtype=np.int32)
test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_ns}, y=test_labels_array,
                    batch_size=128, num_epochs=1, shuffle=False
                )

eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-13-02:11:59
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/68/1f0ftm9d7_b6h52231rtg7mh0000gn/T/tmpy16h5yus/model.ckpt-4190
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-02:11:59
INFO:tensorflow:Saving dict for global step 4190: accuracy = 0.122387156, cross_entropy_loss = 4.0735126, global_step = 4190, loss = 5.7586746
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4190: /var/folders/68/1f0ftm9d7_b6h52231rtg7mh0000gn/T/tmpy16h5yus/model.ckpt-4190


In [143]:
print(eval_metrics)

{'accuracy': 0.122387156, 'cross_entropy_loss': 4.0735126, 'loss': 5.7586746, 'global_step': 4190}
